In [1]:
import os#
os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH\\')
#os.chdir("/rds/general/user/ma11115/home/STK_Search/STK_search")
from pathlib import Path#

import pandas as pd
import torch
import numpy as np
from stk_search.utils.config_utils import read_config, save_config
from stk_search.utils import update_datasets
import importlib
importlib.reload(update_datasets)

<module 'stk_search.utils.update_datasets' from 'C:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_search\\src\\stk_search\\utils\\update_datasets.py'>

In [9]:
# run a search experiment with the new target
from stk_search.utils import run_hpc_utils
importlib.reload(run_hpc_utils)
case_name = 'BO_precursor'#'BO_learned'
df_total_path = "data/output/Full_dataset/New_target200324.csv"
df_precursors_path ="data/output/Prescursor_data/precursor_data_200324.pkl"
exp_name = "new_target"
search_space_loc = "data/input/search_space/test/search_space1.pkl"
which_acquisition = "EI"

string_to_run_notbook, script_qsub = run_hpc_utils.generate_string_run(
                case=case_name,
                target="target",
                aim='maximise',
                num_elem_initialisation=50,
                num_iteration=2,
                df_path=df_total_path,
                benchmark=True,
                test_name= exp_name,
                df_precursors_path=df_precursors_path,
                frag_properties="selected",
                search_space_loc = search_space_loc,
                which_acquisition = which_acquisition,
            )

%run $string_to_run_notbook

src/dev_scripts/run_search_new.py  --case BO_precursor --test_name new_target --target target --aim maximise --benchmark 1 --num_iteration 2 --num_elem_initialisation 50 --which_acquisition EI --df_path data/output/Full_dataset/New_target200324.csv --df_precursors_path data/output/Prescursor_data/precursor_data_200324.pkl --config_dir /rds/general/user/ma11115/home/Geom3D/Geom3D/training/SchNet_frag/ --search_space_loc data/input/search_space/test/search_space1.pkl --frag_properties selected --lim_counter 10
 number of fragment 6
True benchmark


C:\Users\ma11115\OneDrive - Imperial College London\github_folder\STK_search\src\dev_scripts\run_search_new.py:50: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(df_path)


BO_precursor   case  
Index(['Atom_num', 'HOMO (eV)', 'LUMO (eV)', 'HOMO_LUMO_GAP (eV)', 'IP (eV)',
       'EA (eV)', '1 Excited state Energy (eV)', '1 Excited state Osc',
       '2 Excited state Energy (eV)', '2 Excited state Osc',
       '3 Excited state Energy (eV)', '3 Excited state Osc'],
      dtype='object')
Inputs saved.
element Inchikey suggested: RNEXACFTXLDVTO-UHFFFAOYSA-N, Eval: -1.843057149439776
element Inchikey suggested: CRKWPXOHLUHPDS-MPEKYDAASA-N, Eval: -0.2930321810884516
element Inchikey suggested: CGHSNKMDTZDTHF-UHFFFAOYSA-N, Eval: -1.159750321274828
element Inchikey suggested: MIQDDTCZGNSEGN-HQILZYGMSA-N, Eval: -2.019325861192966
element Inchikey suggested: ACLMODMVFZEZMT-UHFFFAOYSA-N, Eval: -1.808525167293411
element Inchikey suggested: HMYIYITXEFTEJJ-DBOAKCCCSA-N, Eval: -4.0657
element Inchikey suggested: MYTLDRNGXNJFLN-UHFFFAOYSA-N, Eval: -0.5811162158320039
element Inchikey suggested: QEGFSLZXQATGCH-OKCVXOCRSA-N, Eval: -1.196281792020672
element Inchikey sugge

In [5]:
import stko
stko.cal